# Integrating CrewAI Tools with the AG2 Framework

In this tutorial, we demonstrate how to integrate [CrewAI Tools](https://github.com/crewAIInc/crewAI-tools/tree/main) into the AG2 framework. This process enables smooth interoperability between the two systems, allowing developers to leverage CrewAI's powerful tools within AG2's flexible agent-based architecture. By the end of this guide, you will understand how to configure agents, convert CrewAI tools for use in AG2, and validate the integration with a practical example.


## Installation
Install the required packages for integrating CrewAI tools into the AG2 framework.
This ensures all dependencies for both frameworks are installed.

```bash
pip install ag2[interop-crewai]
```

## Imports

Import necessary modules and tools.
- `os` is used to access environment variables.
- `Path` helps in handling file paths.
- `TemporaryDirectory` is used for creating a temporary workspace.
- `FileWriterTool` and `ScrapeWebsiteTool` are the CrewAI tools we will integrate.
- `AssistantAgent` and `UserProxyAgent` are core AG2 classes.
- `CrewAIInteroperability` facilitates the interoperability between AG2 and CrewAI.

In [1]:
import os
from pathlib import Path
from tempfile import TemporaryDirectory

from crewai_tools import FileWriterTool, ScrapeWebsiteTool

from autogen import AssistantAgent, UserProxyAgent
from autogen.interoperability.crewai import CrewAIInteroperability

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-16 22:31:29,213 - 139887720012672 - __init__.py-__init__:18 - WARNING: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


## Agent Configuration

Configure the agents for the interaction.
- `config_list` defines the LLM configurations, including the model and API key.
- `UserProxyAgent` simulates user inputs without requiring actual human interaction (set to `NEVER`).
- `AssistantAgent` represents the AI agent, configured with the LLM settings.

In [2]:
config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
)

chatbot = AssistantAgent(
    name="chatbot",
    llm_config={"config_list": config_list},
)

## Tool Integration

Initialize and register the CrewAI tool with AG2.
- `crewai_tool` is an instance of the `FileWriterTool` from CrewAI.
- `CrewAIInteroperability` converts the CrewAI tool to make it usable in AG2.
- `register_for_execution` and `register_for_llm` allow the tool to work with the UserProxyAgent and AssistantAgent.

In [3]:
crewai_tool = FileWriterTool()
crewai_interop = CrewAIInteroperability()
ag2_tool = crewai_interop.convert_tool(crewai_tool)

ag2_tool.register_for_execution(user_proxy)
ag2_tool.register_for_llm(chatbot)

## File creation

Demonstrate the integration by writing to a file using the converted CrewAI tool.
- A temporary directory is created to simulate a file operation environment.
- The `message` instructs the chatbot to use the tool to write a specific string into a file.
- `user_proxy.initiate_chat` starts the interaction, with the chatbot processing the request and using the tool.
- Finally, the output file is verified to ensure the integration works correctly.

In [4]:
with TemporaryDirectory() as tmpdirname:
    filename = "tool_result.txt"
    message = f"""Write 'Easy Migration :)' into {filename}.
Use {tmpdirname} dir.
"""

    user_proxy.initiate_chat(recipient=chatbot, message=message, max_turns=2)

    assert Path(tmpdirname, filename).read_text() == "Easy Migration :)"

User (to chatbot):

Write 'Easy Migration :)' into tool_result.txt.
Use /tmp/tmpt6zkd1db dir.


--------------------------------------------------------------------------------
chatbot (to User):

***** Suggested tool call (call_UR57tA4Dr5YF0aLInnRGa8eD): File_Writer_Tool *****
Arguments: 
{"args":{"filename":"tool_result.txt","directory":"/tmp/tmpt6zkd1db","content":"Easy Migration :)","overwrite":"True"}}
*********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION File_Writer_Tool...
Using Tool: File Writer Tool
User (to chatbot):

***** Response from calling tool (call_UR57tA4Dr5YF0aLInnRGa8eD) *****
Content successfully written to /tmp/tmpt6zkd1db/tool_result.txt
**********************************************************************

--------------------------------------------------------------------------------
chatbot (to User):

The message 'Easy 

## Scrape the web

Demonstrate scraping a website using the converted CrewAI tool.
- The `message` instructs the chatbot to scrape the AG2 website.
- `user_proxy.initiate_chat` starts the interaction, with the chatbot processing the request and using the tool.
- Finally, we output the summary from the LLM

In [5]:
crewai_scrape_tool = ScrapeWebsiteTool()
ag2_tool = crewai_interop.convert_tool(crewai_scrape_tool)

ag2_tool.register_for_execution(user_proxy)
ag2_tool.register_for_llm(chatbot)

message = "Scape the website https://ag2.ai/"

chat_result = user_proxy.initiate_chat(recipient=chatbot, message=message, max_turns=2)

User (to chatbot):

Scape the website https://ag2.ai/

--------------------------------------------------------------------------------
chatbot (to User):

***** Suggested tool call (call_jwhEriZ8xz0sJfbNXZQYeWRH): Read_website_content *****
Arguments: 
{"args":{"website_url":"https://ag2.ai/"}}
*************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION Read_website_content...
Using Tool: Read website content
User (to chatbot):

***** Response from calling tool (call_jwhEriZ8xz0sJfbNXZQYeWRH) *****

AgentOS
Join our growing community of over 20,000 agent builders Join our growing community of over 20,000 agent builders The Open-Source AgentOS Build production-ready multi-agent systems in minutes, not months. Github Discord The End-to-End Platform for Multi-Agent Automation The End-to-End Platform for Multi-Agent Automation Flexible Agent Construction

In [6]:
print(chat_result.summary)

I have scraped the content from the website https://ag2.ai/. Here are the key points:

---

**AgentOS**
- Join a community of over 20,000 agent builders.
- Build production-ready multi-agent systems quickly.
  
**Features:**
- **Flexible Agent Construction and Orchestration**: Define roles, configure behaviors, and orchestrate collaboration through intuitive code.
  - Assistant agents for problem-solving.
  - Executor agents for taking action.
  - Critic agents for validation.
  - Group chat managers for coordination.

- **Built-in Conversation Patterns**: Automatically handles message routing, state management, and conversation flow.
  - Two-agent conversations.
  - Group chats with dynamic speaker selection.
  - Sequential chats with context carryover.
  - Nested conversations for modularity.

- **Seamless Human-AI Collaboration**: Integrate human oversight and input into workflows.
  - Configurable human input modes.
  - Flexible intervention points.
  - Optional human approval work